In [1]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r"D:\Downloads\List 5% PB Stocks\Original Files\financial-data20182021.csv")

In [3]:
df['ID']=df.apply(lambda x:'%s%s%s' % (x['code'],x['year'],x['quarter']),axis=1)
df[df["code"].str.contains("TV2|NSC")==False]
df.head()

,Unnamed: 0,code,quarter,year,net profit after tax,net cash flow from operating activities,revenue,total equity,ID
0,0,AAA,1,2018,60010,-329804,1298292,1512943,AAA20181
1,1,AAA,2,2018,50703,136572,2321958,2889363,AAA20182
2,2,AAA,3,2018,47910,109038,2188615,2927114,AAA20183
3,3,AAA,4,2018,53526,122735,2209962,2980250,AAA20184
4,4,AAA,1,2019,223625,-218406,2616643,3124273,AAA20191


In [4]:
# Step 2
df2 = pd.read_csv(r"D:\Downloads\List 5% PB Stocks\Original Files\price-oi-eod.csv",parse_dates=True)
df2[df2["code"].str.contains("TV2|NSC")==False]
#df2=df2.drop("Unnamed: 0", axis=1)

df2.head()

,code,date,close,oi
0,AAA,8/21/2018,14.52,167.199966
1,AAA,8/22/2018,15.19,167.199966
2,AAA,8/23/2018,15.14,167.199966
3,AAA,8/24/2018,15.23,167.199966
4,AAA,8/27/2018,15.10,167.199966


In [5]:
df2['query'] = (pd.to_datetime(df2['date']).dt.date - relativedelta(months=5)).apply(lambda x: str(x.year) + str(ceil(x.month/3)))
df2['ID'] = df2['code'] + df2['query']
df2=df2[(df2['code']!='TV2')&(df2['code']!='NSC')]

In [6]:
df2

,code,date,close,oi,query,ID
0,AAA,8/21/2018,14.52,167.199966,20181,AAA20181
1,AAA,8/22/2018,15.19,167.199966,20181,AAA20181
2,AAA,8/23/2018,15.14,167.199966,20181,AAA20181
3,AAA,8/24/2018,15.23,167.199966,20181,AAA20181
4,AAA,8/27/2018,15.10,167.199966,20181,AAA20181
...,...,...,...,...,...,...
251896,YEG,7/19/2021,15.80,31.279968,20211,YEG20211
251897,YEG,7/20/2021,15.90,31.279968,20211,YEG20211
251898,YEG,7/21/2021,16.35,31.279968,20211,YEG20211
251899,YEG,7/22/2021,16.35,31.279968,20211,YEG20211


df2['newdate']=pd.to_datetime(df2['date']).dt.date - relativedelta(months=5)
df2=df2.drop(df2.columns[1], axis=1)
df2.head()

df2['newdate'] = pd.to_datetime(df2['newdate'])
df2['quarter'] = df2['newdate'].dt.quarter
df2['year'] = df2['newdate'].dt.year
df2.head()

df2['ID']=df2.apply(lambda x:'%s%s%s' % (x['code'],x['year'],x['quarter']),axis=1)
df2.head()

df2a=df2.groupby('ID').mean()
df2a.head()

df3= pd.merge(df, df2, 
                   on='ID', 
                   how='right')
#df4=df3.drop("Unnamed: 0", axis=1)
#df4=df3.drop(df3.columns[[0,1,2,3,11,12]], axis=1)
df3

In [7]:
df3 = pd.merge(df2, df, on =['ID','code'], how = 'left')
df3.loc[df3['total equity']>0,'pb'] = df3['close']/(df3['total equity']/(df3['oi']*1000))

In [9]:
df3.drop(columns='Unnamed: 0', inplace=True)
df3

,code,date,close,oi,query,ID,quarter,year,net profit after tax,net cash flow from operating activities,revenue,total equity,pb
0,AAA,8/21/2018,14.52,167.199966,20181,AAA20181,1.0,2018.0,60010.0,-329804.0,1298292.0,1512943.0,1.604650
1,AAA,8/22/2018,15.19,167.199966,20181,AAA20181,1.0,2018.0,60010.0,-329804.0,1298292.0,1512943.0,1.678693
2,AAA,8/23/2018,15.14,167.199966,20181,AAA20181,1.0,2018.0,60010.0,-329804.0,1298292.0,1512943.0,1.673168
3,AAA,8/24/2018,15.23,167.199966,20181,AAA20181,1.0,2018.0,60010.0,-329804.0,1298292.0,1512943.0,1.683114
4,AAA,8/27/2018,15.10,167.199966,20181,AAA20181,1.0,2018.0,60010.0,-329804.0,1298292.0,1512943.0,1.668747
...,...,...,...,...,...,...,...,...,...,...,...,...,...
250444,YEG,7/19/2021,15.80,31.279968,20211,YEG20211,1.0,2021.0,-52520.0,-901.0,289531.0,827797.0,0.597035
250445,YEG,7/20/2021,15.90,31.279968,20211,YEG20211,1.0,2021.0,-52520.0,-901.0,289531.0,827797.0,0.600813
250446,YEG,7/21/2021,16.35,31.279968,20211,YEG20211,1.0,2021.0,-52520.0,-901.0,289531.0,827797.0,0.617818
250447,YEG,7/22/2021,16.35,31.279968,20211,YEG20211,1.0,2021.0,-52520.0,-901.0,289531.0,827797.0,0.617818


x=df4['total equity']/df4['oi']
df4['P/B']=df4['close']/x
df4.set_index("ID", inplace = True)
df4.head()

In [10]:
rank_df = pd.pivot_table(df3, values = 'pb', index = ['date'], columns = 'code')

In [12]:
rank_df = rank_df.rank(axis=1, method='first', ascending=True)

In [13]:
rank_df

code,AAA,AAM,ABT,ACC,ACL,ADS,AGF,AGM,AMD,ANV,...,VPS,VRC,VRE,VSC,VSH,VSI,VTB,VTO,YBM,YEG
date,,,,,,,,,,,,,,,,,,,,,
1/10/2019,154.0,33.0,215.0,103.0,121.0,111.0,35.0,69.0,18.0,291.0,...,132.0,198.0,307.0,183.0,226.0,259.0,134.0,50.0,207.0,330.0
1/10/2020,124.0,128.0,199.0,111.0,45.0,133.0,157.0,60.0,5.0,232.0,...,167.0,62.0,321.0,144.0,259.0,250.0,139.0,71.0,39.0,189.0
1/11/2019,154.0,32.0,211.0,102.0,117.0,104.0,30.0,63.0,18.0,294.0,...,141.0,195.0,307.0,185.0,225.0,257.0,145.0,50.0,197.0,330.0
1/11/2021,136.0,66.0,124.0,255.0,174.0,93.0,161.0,60.0,12.0,207.0,...,190.0,29.0,312.0,232.0,210.0,169.0,67.0,57.0,54.0,212.0
1/12/2021,130.0,62.0,120.0,254.0,181.0,88.0,161.0,59.0,13.0,206.0,...,173.0,29.0,313.0,243.0,207.0,164.0,67.0,56.0,58.0,211.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9/7/2018,180.0,29.0,164.0,88.0,17.0,124.0,20.0,53.0,37.0,248.0,...,161.0,257.0,292.0,178.0,219.0,268.0,142.0,56.0,NaN,330.0
9/7/2020,149.0,106.0,178.0,79.0,227.0,118.0,NaN,110.0,15.0,189.0,...,191.0,25.0,304.0,162.0,210.0,231.0,69.0,73.0,44.0,257.0
9/8/2020,150.0,106.0,181.0,79.0,225.0,111.0,NaN,112.0,15.0,197.0,...,187.0,25.0,305.0,163.0,209.0,235.0,75.0,72.0,46.0,258.0


# Requirement 2

In [ ]:
df5 = pd.read_csv(r"C:\Users\trinh\Downloads\Forward_Test_Dataset (1)\Forward_Test_Dataset\financial-data20182021.csv")
df5.head()

In [ ]:
df2. head()

In [ ]:
#for i in range(df2.shape[0]):
    #row = df2.iloc[[0]]
  #      if (df2['code'] == df5['code'] and df2['quarter']==df5['quarter'] and df2['year'] == df5['year']):
   #         x = df5['total equity']/df2['oi']
     #       df2['P/B']=df2['close']/x
     #   else:
      #      row =  df2.iloc[[0]] + 1
#df2. head()
